In [6]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import os
import sys
sys.path.append(os.path.abspath('..'))

from constants import (
    TARGET_SPARSITY_LOW, TARGET_SPARSITY_MID, TARGET_SPARSITY_HIGH,
    BATCH_SIZE_CNN, BATCH_SIZE_VIT, BATCH_SIZE_LLM,
    EPOCHS_SMALL_MODEL, EPOCHS_LARGE_MODEL, EPOCHS_VIT
)
from utils import get_device, get_num_workers, load_weights, print_statistics
from unstructured_pruning import check_model_sparsity, check_sparsity_distribution
from trainer import TrainingArguments, Trainer
from bacp import BaCPTrainingArguments, BaCPTrainer

from datasets.utils.logging import disable_progress_bar
disable_progress_bar()
os.environ["HF_DATASETS_CACHE"] = "/dbfs/hf_datasets"
os.environ["TOKENIZERS_PARALLELISM"] = "false" 


In [8]:
DEVICE = get_device()
NUM_WORKERS = get_num_workers()
print("Using device:", DEVICE)
print("Using", NUM_WORKERS, "workers")

Using device: cuda
Using 288 workers


In [9]:
# Notebook specific variables
MODEL_NAME = 'vgg19'
MODEL_TASK = 'cifar10'
TRAIN = True

## Baseline Accuracy

In [10]:
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.01),
    scheduler_type='linear_with_warmup',
    epochs=100,
    learning_type="baseline",
    patience=50,
    db=False,
)
trainer = Trainer(training_args=training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: 32
[TRAINER] Initialized models
[TRAINER] Optimizer type w/ learning rate: (sgd, 0.01)
[CV DATALOADERS] Loaded cifar10 with splits: ['train', 'validation', 'test']
[TRAINER] Data Initialized for model task: cifar10
[TRAINER] Batch size: 512
[TRAINER] Number of dataloders: 3
[TRAINER] Linear scheduler initialized with warmup steps: 830 and total steps: 8300
[TRAINER] Pruning not initialized
[TRAINER] Saving model to: ./research/vgg19/cifar10/vgg19_cifar10_baseline.pt
[LOGGER] Log file created at location: ./log_records/vgg19/cifar10/baseline/run_1.log
[TRAINER] Training with mixed precision enabled
[TRAINER] Initial model sparsity: 0.0


Epoch [1/100]: Avg Loss: 1.8159 | Avg Accuracy: 61.87 | Model Sparsity: 0.0

[TRAINER] weights saved!


Epoch [2/100]: Avg Loss: 0.9360 | Avg Accuracy: 74.53 | Model Sparsity: 0.0

[TRAINER] weights saved!


Epoch [3/100]: Avg Loss: 0.7061 | Avg Accuracy: 79.16 | Model Sparsity: 0.0

[TRAINER] weights saved!


Epoch [4/100]: Avg Loss: 0.5947 | Avg Accuracy: 81.70 | Model Sparsity: 0.0

[TRAINER] weights saved!


...

Epoch [97/100]: Avg Loss: 0.0019 | Avg Accuracy: 92.15 | Model Sparsity: 0.0



Epoch [98/100]: Avg Loss: 0.0019 | Avg Accuracy: 91.66 | Model Sparsity: 0.0



Epoch [99/100]: Avg Loss: 0.0018 | Avg Accuracy: 91.84 | Model Sparsity: 0.0



Epoch [100/100]: Avg Loss: 0.0020 | Avg Accuracy: 91.81 | Model Sparsity: 0.0

[TRAINER] Loading weights: ./research/vgg19/cifar10/vgg19_cifar10_baseline.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.0



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     91.78%

Model Information:
------------------------------
  Total Parameters:     139,611,210
  Trainable Parameters: 139,611,210
  Model Sparsity:       0.0000 (0.00%)

Training Configuration:
------------------------------
  Model:                vgg19
  Task:                 cifar10
  Learning Type:        baseline
  Batch Size:           512
  Learning Rate:        0.01
  Optimizer:            sgd
  Epochs:               100

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



## Pruning Accuracy

### Magnitude Pruning

In [11]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.01),
    pruning_type="magnitude_pruning",
    target_sparsity=TARGET_SPARSITY_LOW,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: 32
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/vgg19/cifar10/vgg19_cifar10_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (sgd, 0.01)
[CV DATALOADERS] Loaded cifar10 with splits: ['train', 'validation', 'test']
[TRAINER] Data Initialized for model task: cifar10
[TRAINER] Batch size: 512
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
[TRAINER] Pruning initialized
[TRAINER] Pruning type: magnitude_pruning
[TRAINER] Target sparsity: 0.95
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/vgg19/cifar10/vgg19_cifar10_magnitude_pruning_0.95_pruning.pt
[LOGGER] Log file created at location: ./log_records/vgg19/cifar10/pruning/magnitude_pruning/0.95/run_1.log
[TRAINER] Training with mixed precision enabled
[TRAINER] Initial model sparsity: 0.0


Training Epoch [1/5]:   0%|          | 0/83 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.464.



Epoch [1/5]: Avg Loss: 0.0293 | Avg Accuracy: 88.98 | Model Sparsity: 0.4636

[TRAINER] weights saved!


Recovery epoch [1/10]: Avg Loss: 0.0531 | Avg Accuracy: 90.03 | Model Sparsity: 0.4636

[TRAINER] weights saved!


...

Recovery epoch [7/10]: Avg Loss: 0.0200 | Avg Accuracy: 90.78 | Model Sparsity: 0.95



Recovery epoch [8/10]: Avg Loss: 0.0163 | Avg Accuracy: 90.82 | Model Sparsity: 0.95



Recovery epoch [9/10]: Avg Loss: 0.0199 | Avg Accuracy: 91.21 | Model Sparsity: 0.95

[TRAINER] weights saved!


Recovery epoch [10/10]: Avg Loss: 0.0155 | Avg Accuracy: 91.02 | Model Sparsity: 0.95

[TRAINER] Loading weights: ./research/vgg19/cifar10/vgg19_cifar10_magnitude_pruning_0.95_pruning.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.95



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     91.45%

Model Information:
------------------------------
  Total Parameters:     139,611,210
  Trainable Parameters: 139,611,210
  Model Sparsity:       0.9500 (95.00%)

Training Configuration:
------------------------------
  Model:                vgg19
  Task:                 cifar10
  Learning Type:        pruning
  Batch Size:           512
  Learning Rate:        0.01
  Optimizer:            sgd
  Epochs:               5

Pruning Configuration:
------------------------------
  Pruning Type:         magnitude_pruning
  Target Sparsity:      0.95
  Sparsity Scheduler:   cubic
  Recovery Epochs:      10

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



In [13]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.01),
    pruning_type="magnitude_pruning",
    target_sparsity=TARGET_SPARSITY_MID,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if True:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: 32
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/vgg19/cifar10/vgg19_cifar10_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (sgd, 0.01)
[CV DATALOADERS] Loaded cifar10 with splits: ['train', 'validation', 'test']
[TRAINER] Data Initialized for model task: cifar10
[TRAINER] Batch size: 512
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
[TRAINER] Pruning initialized
[TRAINER] Pruning type: magnitude_pruning
[TRAINER] Target sparsity: 0.97
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/vgg19/cifar10/vgg19_cifar10_magnitude_pruning_0.97_pruning.pt
[LOGGER] Log file created at location: ./log_records/vgg19/cifar10/pruning/magnitude_pruning/0.97/run_1.log
[TRAINER] Training with mixed precision enabled
[TRAINER] Initial model sparsity: 0.0


Training Epoch [1/5]:   0%|          | 0/83 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.473.



Epoch [1/5]: Avg Loss: 0.0394 | Avg Accuracy: 90.15 | Model Sparsity: 0.4734

[TRAINER] weights saved!


Recovery epoch [1/10]: Avg Loss: 0.0439 | Avg Accuracy: 89.80 | Model Sparsity: 0.4734



...

Recovery epoch [7/10]: Avg Loss: 0.0291 | Avg Accuracy: 90.97 | Model Sparsity: 0.97



Recovery epoch [8/10]: Avg Loss: 0.0283 | Avg Accuracy: 90.86 | Model Sparsity: 0.97



Recovery epoch [9/10]: Avg Loss: 0.0320 | Avg Accuracy: 91.16 | Model Sparsity: 0.97

[TRAINER] weights saved!


Recovery epoch [10/10]: Avg Loss: 0.0272 | Avg Accuracy: 90.61 | Model Sparsity: 0.97

[TRAINER] Loading weights: ./research/vgg19/cifar10/vgg19_cifar10_magnitude_pruning_0.97_pruning.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.97



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     91.63%

Model Information:
------------------------------
  Total Parameters:     139,611,210
  Trainable Parameters: 139,611,210
  Model Sparsity:       0.9700 (97.00%)

Training Configuration:
------------------------------
  Model:                vgg19
  Task:                 cifar10
  Learning Type:        pruning
  Batch Size:           512
  Learning Rate:        0.01
  Optimizer:            sgd
  Epochs:               5

Pruning Configuration:
------------------------------
  Pruning Type:         magnitude_pruning
  Target Sparsity:      0.97
  Sparsity Scheduler:   cubic
  Recovery Epochs:      10

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



In [14]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.01),
    pruning_type="magnitude_pruning",
    target_sparsity=TARGET_SPARSITY_HIGH,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: 32
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/vgg19/cifar10/vgg19_cifar10_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (sgd, 0.01)
[CV DATALOADERS] Loaded cifar10 with splits: ['train', 'validation', 'test']
[TRAINER] Data Initialized for model task: cifar10
[TRAINER] Batch size: 512
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
[TRAINER] Pruning initialized
[TRAINER] Pruning type: magnitude_pruning
[TRAINER] Target sparsity: 0.99
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/vgg19/cifar10/vgg19_cifar10_magnitude_pruning_0.99_pruning.pt
[LOGGER] Log file created at location: ./log_records/vgg19/cifar10/pruning/magnitude_pruning/0.99/run_1.log
[TRAINER] Training with mixed precision enabled
[TRAINER] Initial model sparsity: 0.0


Training Epoch [1/5]:   0%|          | 0/83 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.483.



Epoch [1/5]: Avg Loss: 0.0358 | Avg Accuracy: 89.06 | Model Sparsity: 0.4831

[TRAINER] weights saved!


Recovery epoch [1/10]: Avg Loss: 0.0646 | Avg Accuracy: 90.43 | Model Sparsity: 0.4831

[TRAINER] weights saved!


...

Recovery epoch [7/10]: Avg Loss: 0.1091 | Avg Accuracy: 90.49 | Model Sparsity: 0.99



Recovery epoch [8/10]: Avg Loss: 0.1057 | Avg Accuracy: 90.99 | Model Sparsity: 0.99

[TRAINER] weights saved!


Recovery epoch [9/10]: Avg Loss: 0.1124 | Avg Accuracy: 90.37 | Model Sparsity: 0.99



Recovery epoch [10/10]: Avg Loss: 0.1045 | Avg Accuracy: 89.19 | Model Sparsity: 0.99

[TRAINER] Loading weights: ./research/vgg19/cifar10/vgg19_cifar10_magnitude_pruning_0.99_pruning.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.99



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     90.67%

Model Information:
------------------------------
  Total Parameters:     139,611,210
  Trainable Parameters: 139,611,210
  Model Sparsity:       0.9900 (99.00%)

Training Configuration:
------------------------------
  Model:                vgg19
  Task:                 cifar10
  Learning Type:        pruning
  Batch Size:           512
  Learning Rate:        0.01
  Optimizer:            sgd
  Epochs:               5

Pruning Configuration:
------------------------------
  Pruning Type:         magnitude_pruning
  Target Sparsity:      0.99
  Sparsity Scheduler:   cubic
  Recovery Epochs:      10

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



### Movement Pruning

In [15]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.01),
    pruning_type="snip_pruning",
    target_sparsity=TARGET_SPARSITY_LOW,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: 32
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/vgg19/cifar10/vgg19_cifar10_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (sgd, 0.01)
[CV DATALOADERS] Loaded cifar10 with splits: ['train', 'validation', 'test']
[TRAINER] Data Initialized for model task: cifar10
[TRAINER] Batch size: 512
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
[TRAINER] Pruning initialized
[TRAINER] Pruning type: snip_pruning
[TRAINER] Target sparsity: 0.95
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/vgg19/cifar10/vgg19_cifar10_snip_pruning_0.95_pruning.pt
[LOGGER] Log file created at location: ./log_records/vgg19/cifar10/pruning/snip_pruning/0.95/run_1.log
[TRAINER] Training with mixed precision enabled
[TRAINER] Initial model sparsity: 0.0


Training Epoch [1/5]:   0%|          | 0/83 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.464.



Epoch [1/5]: Avg Loss: 0.0403 | Avg Accuracy: 89.09 | Model Sparsity: 0.4636

[TRAINER] weights saved!


Recovery epoch [1/10]: Avg Loss: 0.0669 | Avg Accuracy: 89.20 | Model Sparsity: 0.4636

[TRAINER] weights saved!


...

Recovery epoch [7/10]: Avg Loss: 0.0314 | Avg Accuracy: 90.40 | Model Sparsity: 0.95



Recovery epoch [8/10]: Avg Loss: 0.0318 | Avg Accuracy: 90.28 | Model Sparsity: 0.95



Recovery epoch [9/10]: Avg Loss: 0.0352 | Avg Accuracy: 90.50 | Model Sparsity: 0.95



Recovery epoch [10/10]: Avg Loss: 0.0288 | Avg Accuracy: 90.51 | Model Sparsity: 0.95

[TRAINER] Loading weights: ./research/vgg19/cifar10/vgg19_cifar10_snip_pruning_0.95_pruning.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.95



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     91.53%

Model Information:
------------------------------
  Total Parameters:     139,611,210
  Trainable Parameters: 139,611,210
  Model Sparsity:       0.9500 (95.00%)

Training Configuration:
------------------------------
  Model:                vgg19
  Task:                 cifar10
  Learning Type:        pruning
  Batch Size:           512
  Learning Rate:        0.01
  Optimizer:            sgd
  Epochs:               5

Pruning Configuration:
------------------------------
  Pruning Type:         snip_pruning
  Target Sparsity:      0.95
  Sparsity Scheduler:   cubic
  Recovery Epochs:      10

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



In [16]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.01),
    pruning_type="snip_pruning",
    target_sparsity=TARGET_SPARSITY_MID,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: 32
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/vgg19/cifar10/vgg19_cifar10_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (sgd, 0.01)
[CV DATALOADERS] Loaded cifar10 with splits: ['train', 'validation', 'test']
[TRAINER] Data Initialized for model task: cifar10
[TRAINER] Batch size: 512
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
[TRAINER] Pruning initialized
[TRAINER] Pruning type: snip_pruning
[TRAINER] Target sparsity: 0.97
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/vgg19/cifar10/vgg19_cifar10_snip_pruning_0.97_pruning.pt
[LOGGER] Log file created at location: ./log_records/vgg19/cifar10/pruning/snip_pruning/0.97/run_1.log
[TRAINER] Training with mixed precision enabled
[TRAINER] Initial model sparsity: 0.0


Training Epoch [1/5]:   0%|          | 0/83 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.473.



Epoch [1/5]: Avg Loss: 0.0652 | Avg Accuracy: 88.53 | Model Sparsity: 0.4734

[TRAINER] weights saved!


Recovery epoch [1/10]: Avg Loss: 0.0674 | Avg Accuracy: 89.90 | Model Sparsity: 0.4734

[TRAINER] weights saved!


...

Recovery epoch [7/10]: Avg Loss: 0.0680 | Avg Accuracy: 89.80 | Model Sparsity: 0.97



Recovery epoch [8/10]: Avg Loss: 0.0725 | Avg Accuracy: 90.49 | Model Sparsity: 0.97



Recovery epoch [9/10]: Avg Loss: 0.0611 | Avg Accuracy: 90.11 | Model Sparsity: 0.97



Recovery epoch [10/10]: Avg Loss: 0.0625 | Avg Accuracy: 90.03 | Model Sparsity: 0.97

[TRAINER] Loading weights: ./research/vgg19/cifar10/vgg19_cifar10_snip_pruning_0.97_pruning.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.97



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     90.79%

Model Information:
------------------------------
  Total Parameters:     139,611,210
  Trainable Parameters: 139,611,210
  Model Sparsity:       0.9700 (97.00%)

Training Configuration:
------------------------------
  Model:                vgg19
  Task:                 cifar10
  Learning Type:        pruning
  Batch Size:           512
  Learning Rate:        0.01
  Optimizer:            sgd
  Epochs:               5

Pruning Configuration:
------------------------------
  Pruning Type:         snip_pruning
  Target Sparsity:      0.97
  Sparsity Scheduler:   cubic
  Recovery Epochs:      10

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



In [17]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.01),
    pruning_type="snip_pruning",
    target_sparsity=TARGET_SPARSITY_HIGH,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: 32
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/vgg19/cifar10/vgg19_cifar10_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (sgd, 0.01)
[CV DATALOADERS] Loaded cifar10 with splits: ['train', 'validation', 'test']
[TRAINER] Data Initialized for model task: cifar10
[TRAINER] Batch size: 512
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
[TRAINER] Pruning initialized
[TRAINER] Pruning type: snip_pruning
[TRAINER] Target sparsity: 0.99
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/vgg19/cifar10/vgg19_cifar10_snip_pruning_0.99_pruning.pt
[LOGGER] Log file created at location: ./log_records/vgg19/cifar10/pruning/snip_pruning/0.99/run_1.log
[TRAINER] Training with mixed precision enabled
[TRAINER] Initial model sparsity: 0.0


Training Epoch [1/5]:   0%|          | 0/83 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.483.



Epoch [1/5]: Avg Loss: 0.0646 | Avg Accuracy: 89.69 | Model Sparsity: 0.4831

[TRAINER] weights saved!


Recovery epoch [1/10]: Avg Loss: 0.0750 | Avg Accuracy: 90.22 | Model Sparsity: 0.4831

[TRAINER] weights saved!


...

Recovery epoch [7/10]: Avg Loss: 0.1495 | Avg Accuracy: 89.98 | Model Sparsity: 0.99

[TRAINER] weights saved!


Recovery epoch [8/10]: Avg Loss: 0.1405 | Avg Accuracy: 89.98 | Model Sparsity: 0.99



Recovery epoch [9/10]: Avg Loss: 0.1403 | Avg Accuracy: 90.25 | Model Sparsity: 0.99

[TRAINER] weights saved!


Recovery epoch [10/10]: Avg Loss: 0.1377 | Avg Accuracy: 89.86 | Model Sparsity: 0.99

[TRAINER] Loading weights: ./research/vgg19/cifar10/vgg19_cifar10_snip_pruning_0.99_pruning.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.99



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     89.98%

Model Information:
------------------------------
  Total Parameters:     139,611,210
  Trainable Parameters: 139,611,210
  Model Sparsity:       0.9900 (99.00%)

Training Configuration:
------------------------------
  Model:                vgg19
  Task:                 cifar10
  Learning Type:        pruning
  Batch Size:           512
  Learning Rate:        0.01
  Optimizer:            sgd
  Epochs:               5

Pruning Configuration:
------------------------------
  Pruning Type:         snip_pruning
  Target Sparsity:      0.99
  Sparsity Scheduler:   cubic
  Recovery Epochs:      10

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



### Wanda Pruning

In [18]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.01),
    pruning_type="wanda_pruning",
    target_sparsity=TARGET_SPARSITY_LOW,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: 32
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/vgg19/cifar10/vgg19_cifar10_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (sgd, 0.01)
[CV DATALOADERS] Loaded cifar10 with splits: ['train', 'validation', 'test']
[TRAINER] Data Initialized for model task: cifar10
[TRAINER] Batch size: 512
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
[TRAINER] Pruning initialized
[TRAINER] Pruning type: wanda_pruning
[TRAINER] Target sparsity: 0.95
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/vgg19/cifar10/vgg19_cifar10_wanda_pruning_0.95_pruning.pt
[LOGGER] Log file created at location: ./log_records/vgg19/cifar10/pruning/wanda_pruning/0.95/run_1.log
[TRAINER] Training with mixed precision enabled
[TRAINER] Initial model sparsity: 0.0
[Pruner] Adding hooks


Training Epoch [1/5]:   0%|          | 0/83 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.464.



Training Epoch [1/5]:   5%|▍         | 4/83 [00:02<00:38,  2.07it/s, Loss=0.0305, Sparsity=0.4636]


[Pruner] Removing hooks


...

Recovery epoch [7/10]: Avg Loss: 0.0355 | Avg Accuracy: 90.64 | Model Sparsity: 0.95



Recovery epoch [8/10]: Avg Loss: 0.0292 | Avg Accuracy: 91.17 | Model Sparsity: 0.95

[TRAINER] weights saved!


Recovery epoch [9/10]: Avg Loss: 0.0261 | Avg Accuracy: 91.32 | Model Sparsity: 0.95

[TRAINER] weights saved!


Recovery epoch [10/10]: Avg Loss: 0.0234 | Avg Accuracy: 90.83 | Model Sparsity: 0.95

[TRAINER] Loading weights: ./research/vgg19/cifar10/vgg19_cifar10_wanda_pruning_0.95_pruning.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.95



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     91.54%

Model Information:
------------------------------
  Total Parameters:     139,611,210
  Trainable Parameters: 139,611,210
  Model Sparsity:       0.9500 (95.00%)

Training Configuration:
------------------------------
  Model:                vgg19
  Task:                 cifar10
  Learning Type:        pruning
  Batch Size:           512
  Learning Rate:        0.01
  Optimizer:            sgd
  Epochs:               5

Pruning Configuration:
------------------------------
  Pruning Type:         wanda_pruning
  Target Sparsity:      0.95
  Sparsity Scheduler:   cubic
  Recovery Epochs:      10

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



In [19]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.01),
    pruning_type="wanda_pruning",
    target_sparsity=TARGET_SPARSITY_MID,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: 32
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/vgg19/cifar10/vgg19_cifar10_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (sgd, 0.01)
[CV DATALOADERS] Loaded cifar10 with splits: ['train', 'validation', 'test']
[TRAINER] Data Initialized for model task: cifar10
[TRAINER] Batch size: 512
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
[TRAINER] Pruning initialized
[TRAINER] Pruning type: wanda_pruning
[TRAINER] Target sparsity: 0.97
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/vgg19/cifar10/vgg19_cifar10_wanda_pruning_0.97_pruning.pt
[LOGGER] Log file created at location: ./log_records/vgg19/cifar10/pruning/wanda_pruning/0.97/run_1.log
[TRAINER] Training with mixed precision enabled
[TRAINER] Initial model sparsity: 0.0
[Pruner] Adding hooks


Training Epoch [1/5]:   0%|          | 0/83 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.473.



Training Epoch [1/5]:   4%|▎         | 3/83 [00:02<00:54,  1.48it/s, Loss=0.0451, Sparsity=0.4734]


[Pruner] Removing hooks


...

Recovery epoch [7/10]: Avg Loss: 0.0395 | Avg Accuracy: 90.75 | Model Sparsity: 0.97

[TRAINER] weights saved!


Recovery epoch [8/10]: Avg Loss: 0.0338 | Avg Accuracy: 91.24 | Model Sparsity: 0.97

[TRAINER] weights saved!


Recovery epoch [9/10]: Avg Loss: 0.0323 | Avg Accuracy: 90.44 | Model Sparsity: 0.97



Recovery epoch [10/10]: Avg Loss: 0.0415 | Avg Accuracy: 90.22 | Model Sparsity: 0.97

[TRAINER] Loading weights: ./research/vgg19/cifar10/vgg19_cifar10_wanda_pruning_0.97_pruning.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.97



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     90.57%

Model Information:
------------------------------
  Total Parameters:     139,611,210
  Trainable Parameters: 139,611,210
  Model Sparsity:       0.9700 (97.00%)

Training Configuration:
------------------------------
  Model:                vgg19
  Task:                 cifar10
  Learning Type:        pruning
  Batch Size:           512
  Learning Rate:        0.01
  Optimizer:            sgd
  Epochs:               5

Pruning Configuration:
------------------------------
  Pruning Type:         wanda_pruning
  Target Sparsity:      0.97
  Sparsity Scheduler:   cubic
  Recovery Epochs:      10

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



In [20]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"
training_args = TrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.01),
    pruning_type="wanda_pruning",
    target_sparsity=TARGET_SPARSITY_HIGH,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type="pruning",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: 32
[TRAINER] Initialized models
[TRAINER] Loading weights: ./research/vgg19/cifar10/vgg19_cifar10_baseline.pt
[TRAINER] Weights loaded
[TRAINER] Optimizer type w/ learning rate: (sgd, 0.01)
[CV DATALOADERS] Loaded cifar10 with splits: ['train', 'validation', 'test']
[TRAINER] Data Initialized for model task: cifar10
[TRAINER] Batch size: 512
[TRAINER] Number of dataloders: 3
[TRAINER] No scheduler initialized
[TRAINER] Pruning initialized
[TRAINER] Pruning type: wanda_pruning
[TRAINER] Target sparsity: 0.99
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/vgg19/cifar10/vgg19_cifar10_wanda_pruning_0.99_pruning.pt
[LOGGER] Log file created at location: ./log_records/vgg19/cifar10/pruning/wanda_pruning/0.99/run_1.log
[TRAINER] Training with mixed precision enabled
[TRAINER] Initial model sparsity: 0.0
[Pruner] Adding hooks


Training Epoch [1/5]:   0%|          | 0/83 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.483.



Training Epoch [1/5]:   4%|▎         | 3/83 [00:02<01:03,  1.27it/s, Loss=0.0466, Sparsity=0.4831]


[Pruner] Removing hooks


...

Recovery epoch [7/10]: Avg Loss: 0.0824 | Avg Accuracy: 90.04 | Model Sparsity: 0.99



Recovery epoch [8/10]: Avg Loss: 0.0766 | Avg Accuracy: 89.65 | Model Sparsity: 0.99



Recovery epoch [9/10]: Avg Loss: 0.0797 | Avg Accuracy: 90.22 | Model Sparsity: 0.99



Recovery epoch [10/10]: Avg Loss: 0.0778 | Avg Accuracy: 90.62 | Model Sparsity: 0.99

[TRAINER] weights saved!
[TRAINER] Loading weights: ./research/vgg19/cifar10/vgg19_cifar10_wanda_pruning_0.99_pruning.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.99



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     90.53%

Model Information:
------------------------------
  Total Parameters:     139,611,210
  Trainable Parameters: 139,611,210
  Model Sparsity:       0.9900 (99.00%)

Training Configuration:
------------------------------
  Model:                vgg19
  Task:                 cifar10
  Learning Type:        pruning
  Batch Size:           512
  Learning Rate:        0.01
  Optimizer:            sgd
  Epochs:               5

Pruning Configuration:
------------------------------
  Pruning Type:         wanda_pruning
  Target Sparsity:      0.99
  Sparsity Scheduler:   cubic
  Recovery Epochs:      10

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



## BaCP Accuracies

### Magnitude Pruning

In [21]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.1),
    pruning_type='magnitude_pruning',
    target_sparsity=TARGET_SPARSITY_LOW,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type='bacp_pruning',
    db=False,
)
bacp_trainer = BaCPTrainer(bacp_training_args=bacp_training_args)
if TRAIN:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type_and_lr=('adamw', 0.0001),
    pruner=bacp_trainer.get_pruner(),
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    epochs=50,
    finetuned_weights=bacp_trainer.save_path,
    finetune=True,
    learning_type="bacp_finetune",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: 32
[ERROR] Could not load weights: ./research/vgg19/cifar10/vgg19_cifar10_baseline.pt
[ERROR] Attempting partial load
[TRAINER] Weights loaded successfully
[TRAINER] Initialized BaCP models
[TRAINER] Optimizer type w/ learning rate: (sgd, 0.1)
[TRAINER] No scheduler initialized
[CV DATALOADERS] Loaded cifar10 with splits: ['train', 'validation', 'test']
[TRAINER] Data Initialized for model task: cifar10
[TRAINER] Batch size: 512
[TRAINER] Number of dataloders: 3
[TRAINER] Pruning initialized
[TRAINER] Pruning type: magnitude_pruning
[TRAINER] Target sparsity: 0.95
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/vgg19/cifar10/vgg19_cifar10_magnitude_pruning_0.95_bacp_pruning.pt
[LOGGER] Log file created at location: ./log_records/vgg19/cifar10/bacp_pruning/magnitude_pruning/0.95/run_2.log


Training Epoch [1/5]:   0%|          | 0/83 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.464.



Epoch [1/5]: Avg Total Loss: nan | Avg PrC Loss: nan | Avg SnC Loss: 0.0000 | Avg FiC Loss: nan | Avg CE Loss: nan | Model Sparsity: 0.4636

[BaCP] weights saved!


Retraining Epoch [1/10]: Avg Total Loss: nan | Avg PrC Loss: nan | Avg SnC Loss: 0.0000 | Avg FiC Loss: nan | Avg CE Loss: nan | Model Sparsity: 0.4636



...

Epoch [38/50]: Avg Loss: nan | Avg Accuracy: 10.37 | Model Sparsity: 0.95



Epoch [39/50]: Avg Loss: nan | Avg Accuracy: 10.34 | Model Sparsity: 0.95



Epoch [40/50]: Avg Loss: nan | Avg Accuracy: 10.35 | Model Sparsity: 0.95



Epoch [41/50]: Avg Loss: nan | Avg Accuracy: 10.34 | Model Sparsity: 0.95

[TRAINER] Training stopped. No improvements for 20 epochs.
[TRAINER] Loading weights: ./research/vgg19/cifar10/vgg19_cifar10_magnitude_pruning_0.95_bacp_finetune.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.95



TRAINING STATISTICS SUMMARY

Performance Metrics:
------------------------------
  Accuracy:     10.05%

Model Information:
------------------------------
  Total Parameters:     139,611,210
  Trainable Parameters: 139,611,210
  Model Sparsity:       0.9500 (95.00%)

Training Configuration:
------------------------------
  Model:                vgg19
  Task:                 cifar10
  Learning Type:        bacp_finetune
  Batch Size:           512
  Learning Rate:        0.0001
  Optimizer:            adamw
  Epochs:               50

System Information:
------------------------------
  Device:               cuda
  Mixed Precision:      True
  Workers:              24



In [ ]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.1),
    pruning_type='magnitude_pruning',
    target_sparsity=TARGET_SPARSITY_MID,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type='bacp_pruning',
    db=False,
)
bacp_trainer = BaCPTrainer(bacp_training_args=bacp_training_args)
if TRAIN:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type_and_lr=('adamw', 0.0001),
    pruner=bacp_trainer.get_pruner(),
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    epochs=50,
    finetuned_weights=bacp_trainer.save_path,
    finetune=True,
    learning_type="bacp_finetune",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

[TRAINER] Image size: 32
[ERROR] Could not load weights: ./research/vgg19/cifar10/vgg19_cifar10_baseline.pt
[ERROR] Attempting partial load
[TRAINER] Weights loaded successfully
[TRAINER] Initialized BaCP models
[TRAINER] Optimizer type w/ learning rate: (sgd, 0.1)
[TRAINER] No scheduler initialized
[CV DATALOADERS] Loaded cifar10 with splits: ['train', 'validation', 'test']
[TRAINER] Data Initialized for model task: cifar10
[TRAINER] Batch size: 512
[TRAINER] Number of dataloders: 3
[TRAINER] Pruning initialized
[TRAINER] Pruning type: magnitude_pruning
[TRAINER] Target sparsity: 0.97
[TRAINER] Sparsity scheduler: cubic
[TRAINER] Pruning epochs: 5
[TRAINER] Current sparsity: 0.0000
[TRAINER] Saving model to: ./research/vgg19/cifar10/vgg19_cifar10_magnitude_pruning_0.97_bacp_pruning.pt
[LOGGER] Log file created at location: ./log_records/vgg19/cifar10/bacp_pruning/magnitude_pruning/0.97/run_1.log


Training Epoch [1/5]:   0%|          | 0/83 [00:00<?, ?it/s]


[Pruner] Cubic Sparsity ratio increased to 0.473.



Epoch [1/5]: Avg Total Loss: 8.3869 | Avg PrC Loss: 3.9039 | Avg SnC Loss: 0.0000 | Avg FiC Loss: 3.6791 | Avg CE Loss: 0.8040 | Model Sparsity: 0.4734

[BaCP] weights saved!


Retraining Epoch [1/10]: Avg Total Loss: 7.8512 | Avg PrC Loss: 3.7012 | Avg SnC Loss: 0.0000 | Avg FiC Loss: 3.5793 | Avg CE Loss: 0.5707 | Model Sparsity: 0.4734



...

Epoch [24/50]: Avg Loss: 0.1619 | Avg Accuracy: 91.27 | Model Sparsity: 0.97

[TRAINER] weights saved!


Epoch [25/50]: Avg Loss: 0.1579 | Avg Accuracy: 91.17 | Model Sparsity: 0.97



Epoch [26/50]: Avg Loss: 0.1573 | Avg Accuracy: 90.79 | Model Sparsity: 0.97



Epoch [27/50]: Avg Loss: 0.1517 | Avg Accuracy: 90.79 | Model Sparsity: 0.97



Training Epoch [28/50]:  45%|████▍     | 37/83 [00:01<00:02, 20.99it/s, Loss=0.1402, Sparsity=0.97]

In [ ]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.1),
    pruning_type='magnitude_pruning',
    target_sparsity=TARGET_SPARSITY_HIGH,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type='bacp_pruning',
    db=False,
)
bacp_trainer = BaCPTrainer(bacp_training_args=bacp_training_args)
if TRAIN:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type_and_lr=('adamw', 0.0001),
    pruner=bacp_trainer.get_pruner(),
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    epochs=50,
    finetuned_weights=bacp_trainer.save_path,
    finetune=True,
    learning_type="bacp_finetune",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

### SNIP-it Pruning

In [ ]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.1),
    pruning_type='snip_pruning',
    target_sparsity=TARGET_SPARSITY_LOW,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type='bacp_pruning',
    db=False,
)
bacp_trainer = BaCPTrainer(bacp_training_args=bacp_training_args)
if TRAIN:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type_and_lr=('adamw', 0.0001),
    pruner=bacp_trainer.get_pruner(),
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    epochs=50,
    finetuned_weights=bacp_trainer.save_path,
    finetune=True,
    learning_type="bacp_finetune",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

In [ ]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.1),
    pruning_type='snip_pruning',
    target_sparsity=TARGET_SPARSITY_MID,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type='bacp_pruning',
    db=False,
)
bacp_trainer = BaCPTrainer(bacp_training_args=bacp_training_args)
if TRAIN:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type_and_lr=('adamw', 0.0001),
    pruner=bacp_trainer.get_pruner(),
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    epochs=50,
    finetuned_weights=bacp_trainer.save_path,
    finetune=True,
    learning_type="bacp_finetune",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

In [ ]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.1),
    pruning_type='snip_pruning',
    target_sparsity=TARGET_SPARSITY_HIGH,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type='bacp_pruning',
    db=False,
)
bacp_trainer = BaCPTrainer(bacp_training_args=bacp_training_args)
if TRAIN:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type_and_lr=('adamw', 0.0001),
    pruner=bacp_trainer.get_pruner(),
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    epochs=50,
    finetuned_weights=bacp_trainer.save_path,
    finetune=True,
    learning_type="bacp_finetune",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

### Wanda Pruning

In [ ]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.1),
    pruning_type='wanda_pruning',
    target_sparsity=TARGET_SPARSITY_LOW,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type='bacp_pruning',
    db=False,
)
bacp_trainer = BaCPTrainer(bacp_training_args=bacp_training_args)
if TRAIN:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type_and_lr=('adamw', 0.0001),
    pruner=bacp_trainer.get_pruner(),
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    epochs=50,
    finetuned_weights=bacp_trainer.save_path,
    finetune=True,
    learning_type="bacp_finetune",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

In [ ]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.1),
    pruning_type='wanda_pruning',
    target_sparsity=TARGET_SPARSITY_MID,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type='bacp_pruning',
    db=False,
)
bacp_trainer = BaCPTrainer(bacp_training_args=bacp_training_args)
if TRAIN:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type_and_lr=('adamw', 0.0001),
    pruner=bacp_trainer.get_pruner(),
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    epochs=50,
    finetuned_weights=bacp_trainer.save_path,
    finetune=True,
    learning_type="bacp_finetune",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)

In [ ]:
# Initializing finetuned weights path
finetuned_weights = f"./research/{MODEL_NAME}/{MODEL_TASK}/{MODEL_NAME}_{MODEL_TASK}_baseline.pt"

bacp_training_args = BaCPTrainingArguments(
    model_name=MODEL_NAME,
    model_task=MODEL_TASK,
    batch_size=BATCH_SIZE_CNN,
    optimizer_type_and_lr=('sgd', 0.1),
    pruning_type='wanda_pruning',
    target_sparsity=TARGET_SPARSITY_HIGH,
    sparsity_scheduler='cubic',
    finetuned_weights=finetuned_weights,
    learning_type='bacp_pruning',
    db=False,
)
bacp_trainer = BaCPTrainer(bacp_training_args=bacp_training_args)
if TRAIN:
    bacp_trainer.train()

# Finetuning Phase
bacp_trainer.generate_mask_from_model()
training_args = TrainingArguments(
    model_name=bacp_trainer.model_name,
    model_task=bacp_trainer.model_task,
    batch_size=bacp_trainer.batch_size,
    optimizer_type_and_lr=('adamw', 0.0001),
    pruner=bacp_trainer.get_pruner(),
    pruning_type=bacp_trainer.pruning_type,
    target_sparsity=bacp_trainer.target_sparsity,
    epochs=50,
    finetuned_weights=bacp_trainer.save_path,
    finetune=True,
    learning_type="bacp_finetune",
    db=False,
)
trainer = Trainer(training_args)
if TRAIN:
    trainer.train()

metrics = trainer.evaluate()
print_statistics(metrics, trainer)